### 1. 라이브러리 선언하기

In [1]:
import pandas as pd
import numpy as np

### 2. 데이터 불러오기

selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

#숫자형 데이터 기본 프레임을 보여줌
selloutData.describe()

selloutData.info()

#YEARWEEK 타입 str 로 변경
selloutData["YEARWEEK"] = selloutData.YEARWEEK.astype(str)

# 타입 변경 확인
selloutData.dtypes

# QTY 가 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값을 QTY_NEW 컬럼에 추가 
selloutData["QTY_NEW"] = \
    np.where(selloutData.QTY < 0, 0 , selloutData.QTY)

#QTY 음수값 확인
selloutData.QTY.min()

#QTY 음수 개수 확인
selloutData[selloutData.QTY < 0]["QTY"].count()

#QTY_NEW 최소값 확인
selloutData.QTY_NEW.min()

#QTY_NEW 음수값 개수 확인
selloutData[selloutData.QTY_NEW < 0]["QTY"].count()

selloutData

# YEAR 컬럼 생성
selloutData["YEAR"] = selloutData.YEARWEEK.str[0:4]

# WEEK 컬럼 생성
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:6]

selloutData.dtypes







#WEEK 가 52 이하인 데이터만 조회 후 refinedSelloutData 변수에 담기
refinedSelloutData = selloutData[selloutData.WEEK.astype(int)  <= 52]

refinedSelloutData

#정렬할 컬럼 지정
sortKey = ["REGIONID","PRODUCT","WEEK"]

# 지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여 sortedData 변수에 담음
sortedData=refinedSelloutData.sort_values(by=sortKey,ignore_index=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124658 entries, 0 to 124657
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   REGIONID  124658 non-null  object 
 1   PRODUCT   124658 non-null  object 
 2   YEARWEEK  124658 non-null  int64  
 3   QTY       124658 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.8+ MB


In [2]:
#지역, 상품, 연도 별 평균 판매량의 평균산출
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = sortedData.\
groupby(by=groupKey)["QTY_NEW"].agg(["mean"])

# 컬럼명 변경 (mean -> QTY_MEAN)
groupData.rename(columns={"mean":"QTY_MEAN"}, 
inplace=True)

# 인덱스 초기화
groupData.reset_index(inplace=True)
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [8]:
# refinedSelloutData와 groupData를 [REGIONID, PRODUCT, YEAR] 키로 조인하여 MergedData 변수에 아래와 같이 담으세요

In [3]:
mergedData = pd.merge(left=refinedSelloutData,
        right=groupData,
        on=("REGIONID","PRODUCT","YEAR"),
        how="left")

In [10]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
mergedData.sort_values(by=sortKey,ignore_index=True)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
0,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
1,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
2,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
3,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
4,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538,1.869826
...,...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846,0.858356
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846,1.051445
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815


In [11]:
mergedData["SEASONALITY"] = mergedData["QTY_NEW"]/ mergedData["QTY_MEAN"]

In [12]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
0,A60,PRODUCT4,201402,71.0,71.0,2014,02,29.134615,2.436964
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02,22993.923077,0.968734
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02,6077.653846,0.734329
3,A60,PRODUCT47,201402,0.0,0.0,2014,02,0.288462,0.000000
4,A60,PRODUCT56,201402,23.0,23.0,2014,02,16.730769,1.374713
...,...,...,...,...,...,...,...,...,...
123859,A10,PRODUCT60,201630,824.0,824.0,2016,30,721.923077,1.141396
123860,A10,PRODUCT56,201630,275.0,275.0,2016,30,235.307692,1.168683
123861,A10,PRODUCT61,201630,0.0,0.0,2016,30,0.019231,0.000000
123862,A10,PRODUCT12,201630,15021.0,15021.0,2016,30,29025.711538,0.517507


In [15]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
mergedData.sort_values(by=sortKey,ignore_index=True)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
0,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
1,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
2,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
3,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
4,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538,1.869826
...,...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846,0.858356
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846,1.051445
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815


In [16]:
groupKey = ["REGIONID","PRODUCT","WEEK"]
finalResult = mergedData.\
groupby(by=groupKey)["SEASONALITY"].agg(["mean"])
# groupData.rename(columns={"mean":"QTY_MEAN"}, 
# inplace=True)

In [18]:
finalResult.reset_index(inplace=True)
finalResult

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192
